# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Parse First Text File

In [2]:
file_path = 'mathwell_questions2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100) #past percentage was 54.88
print(result1)
print(len(df1), total_questions)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
Percentage of Questions with Executable Code: 55.52961640574934
12131 21846


In [3]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 6.95738191410436


In [4]:
df1 = df1.drop_duplicates(subset='question')

In [5]:
df1

,question,solution,answer,topic
0,A baker baked 120 cakes. He sold 40 on the fir...,def solution():\n\n # The baker started wit...,30,Animals and Wildlife
1,Naruto has 300 ramen noodles. He eats 20 ramen...,def solution():\n #Naruto has 300 ramen noo...,200,Naruto
2,A 5th-grade class is doing a science project. ...,def solution():\n #20 students are in the c...,60,DIY Science Projects
3,A mummy is 3000 years old. \n\nHis sarcophagus...,def solution():\n #The mummy is 3000 years ...,4000,Mummies
4,The volleyball team has 12 players. \n4 of the...,def solution():\n #Total number of players\...,0,volleyball
...,...,...,...,...
12126,The Paw Patrol is a team of 6 puppies. \n\n2 o...,def solution():\n #The Paw Patrol is a team...,2,Paw Patrol
12127,"The world's population of tigers is 3,90 00. \...",def solution():\n # total tigers in the wor...,1145,Wildlife Conservation
12128,"On Sesame Street, there are 12 Muppets. \n4 of...",def solution():\n #There are 12 Muppets\n ...,1,Sesame Street
12129,A submarine is exploring the ocean. It is 1000...,def solution():\n #A submarine is exploring...,3000,Exploring the Ocean


# Parse Second Text File

In [6]:
file_path = 'mathwell_questions_no_topic.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
print(result1)
print(len(df2), total_questions)

Percentage of Questions with Executable Code: 46.70164272813279
5430 11627


In [7]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 4.953959484346225


In [8]:
df2 = df2.drop_duplicates(subset='question')

In [9]:
df2

,question,solution,answer,topic
0,2000 people in a stadium. 1/4 of the people ar...,def solution():\n #Total people in the stad...,850.0,unspecified
1,The first 500 people in line for the Comic-Con...,def solution():\n #The first 500 people in ...,1000,unspecified
2,There are 120000 spiders in a spider farm. 100...,def solution():\n #There were 120000 spider...,100000,unspecified
3,A baker bakes 100 pies. He sells 20 of them on...,def solution():\n #Baker bakes 100 pies\n ...,10,unspecified
4,A 747 can hold 400 people. \nA 777 can hold 30...,def solution():\n #A 747 can hold 400 peopl...,4000,unspecified
...,...,...,...,...
5417,"A pizzeria sells 20 pizzas on Monday, 15 pizza...",def solution():\n #The pizzeria sold 20 piz...,60,unspecified
5418,10000 people lined up for a free t-shirt. 1000...,def solution():\n #10000 people lined up fo...,8900,unspecified
5420,A cake has 12 slices. \n\n3 of the slices are ...,def solution():\n #A cake has 12 slices\n ...,4,unspecified
5422,The Avengers are a superhero team. \nThey have...,def solution():\n #The Avengers have 10 mem...,2,unspecified


# Concat DFs

In [10]:
extra_qs = pd.read_csv('data/extra_qs.csv')
df = pd.concat([df1, df2, extra_qs])
df = df.drop_duplicates(subset='question')
df = df.sample(frac = 1, random_state = 42)
df = df[['question', 'solution', 'answer', 'topic']]
df.to_csv("data/sgsm_unannotated.csv")

In [11]:
df ## last run had 15670

,question,solution,answer,topic
8475,A stamp collector has 1000 stamps. \n\nHe has ...,def solution():\n\n #A stamp collector has ...,400,"Hobbies (e.g., stamp collecting, coin collecting)"
7020,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman's speed\n su...,0.5,Superman
11650,The wizard has 200 spells. \nHe casts 30 spell...,def solution():\n #The wizard has 200 spell...,135,Magic and Wizards
4652,The 49ers have 250 rushing yards and 200 passi...,def solution():\n\n #The 49ers have 250 rus...,200,unspecified
6329,Naruto has 1000 ryo. 100 ryo is 10% of his mon...,def solution():\n #Naruto has 1000 ryo\n ...,780,Naruto
...,...,...,...,...
5513,A bird watcher is in the forest. He sees 500 b...,def solution():\n #Total number of birds\n ...,600,Birds and Bird Watching
3297,25000 people are at a music festival. \n10000 ...,def solution():\n #25000 people are at a mu...,70000,unspecified
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)"


In [12]:
df['topic'].value_counts()

unspecified                    3405
LeBron James                    160
cheer leading                   157
Captain America                 156
Wonder Woman                    155
                               ... 
Space and Astronauts              9
Hiking and Nature                 8
Star Wars                         7
Transformers                      6
Pets (cats, dogs, hamsters)       3
Name: topic, Length: 198, dtype: int64